In [10]:
import numpy as np
import pandas as pd
import requests
import json
import certifi
from datetime import datetime

There are three stage of programming here:
(1) Retrivieng all the trips options for a single OD and a single date.
(2) Retrieving all the trips options of all the ODs for a single date.
(3) Retrieving all the trips options for all the OD and for all the dates.

Retrieve data from the link

In [3]:
#URL from where I am retrieving the data
url = "https://api.ember.to/v1/quotes/?origin=13&destination=42&departure_date_from=2024-10-24T00%3A00%3A00%2B01%3A00&departure_date_to=2024-10-25T04%3A00%3A00%2B01%3A00&include_unlisted_trips=true"

#Data from requestor
response = requests.get(url, verify=False)

#Check the data
if response.status_code == 200:
    data = response.json()  # Parse the JSON data
    print(json.dumps(data, indent=4))  # Print JSON data
else:
    print(f"Failed to retrieve data: {response.status_code}")

c:\Users\Luis.ParraMorales\AppData\Local\miniforge3\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ember.to'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{
    "quotes": [
        {
            "availability": {
                "seat": 51,
                "wheelchair": 1,
                "bicycle": 2
            },
            "prices": {
                "adult": 850,
                "child": 425,
                "young_child": 0,
                "concession": 0,
                "seat": 0,
                "wheelchair": 0,
                "bicycle": 0
            },
            "legs": [
                {
                    "type": "scheduled_transit",
                    "trip_uid": "oPz4LEFSsVH53UrLLxax3Z",
                    "adds_capacity_for_trip_uid": null,
                    "origin": {
                        "id": 218,
                        "atco_code": "6400LL99",
                        "detailed_name": "Slessor Gardens",
                        "lat": 56.459319,
                        "lon": -2.966036,
                        "name": "Dundee Slessor Gardens",
                        "region_name": "Dundee",
            

EXTRACT DATA FROM 

In [4]:
# List to store the extracted information
quotes_list = []

# Extract and store specific information for each quote
for quote in data['quotes']:
    for leg in quote['legs']:  # Loop through each leg in the legs list
        origin_name = leg['origin']['name']
        origin_code = leg['origin']['code']
        destination_name = leg['destination']['name']
        destination_code = leg['destination']['code']
        departure_date = leg['departure']['scheduled']
        arrival_date = leg['arrival']['scheduled']
        operator = leg['description']['operator']
        fare = quote['prices']['adult']

        # Calculate duration
        departure_time = datetime.fromisoformat(departure_date.replace("Z", "+00:00"))
        arrival_time = datetime.fromisoformat(arrival_date.replace("Z", "+00:00"))
        duration = arrival_time - departure_time

        # Append the extracted information to the list
        quotes_list.append({
            "Origin Name": origin_name,
            "Origin Code": origin_code,
            "Destination Name": destination_name,
            "Destination Code" : destination_code,
            "Departure Date": departure_date,
            "Duration": duration,
            "Operator": operator,
            "Fare": fare
        })

# Create a DataFrame from the list
df = pd.DataFrame(quotes_list)

# Print the DataFrame
print(df)

               Origin Name Origin Code Destination Name Destination Code  \
0   Dundee Slessor Gardens         DUN    George Street              EDI   
1   Dundee Slessor Gardens         DUN    George Street              EDI   
2   Dundee Slessor Gardens         DUN    George Street              EDI   
3   Dundee Slessor Gardens         DUN    George Street              EDI   
4   Dundee Slessor Gardens         DUN    George Street              EDI   
5   Dundee Slessor Gardens         DUN    George Street              EDI   
6   Dundee Slessor Gardens         DUN    George Street              EDI   
7   Dundee Slessor Gardens         DUN    George Street              EDI   
8   Dundee Slessor Gardens         DUN    George Street              EDI   
9   Dundee Slessor Gardens         DUN    George Street              EDI   
10  Dundee Slessor Gardens         DUN    George Street              EDI   
11  Dundee Slessor Gardens         DUN    George Street              EDI   
12  Dundee S

In [5]:
print(df.head(8))

              Origin Name Origin Code Destination Name Destination Code  \
0  Dundee Slessor Gardens         DUN    George Street              EDI   
1  Dundee Slessor Gardens         DUN    George Street              EDI   
2  Dundee Slessor Gardens         DUN    George Street              EDI   
3  Dundee Slessor Gardens         DUN    George Street              EDI   
4  Dundee Slessor Gardens         DUN    George Street              EDI   
5  Dundee Slessor Gardens         DUN    George Street              EDI   
6  Dundee Slessor Gardens         DUN    George Street              EDI   
7  Dundee Slessor Gardens         DUN    George Street              EDI   

              Departure Date        Duration Operator  Fare  
0  2024-10-23T23:45:00+00:00 0 days 01:42:00    Ember   850  
1  2024-10-24T01:45:00+00:00 0 days 01:47:00    Ember   850  
2  2024-10-24T03:30:00+00:00 0 days 01:53:00    Ember   850  
3  2024-10-24T04:05:00+00:00 0 days 01:45:00    Ember   850  
4  2024-10-24T

NOW I AM EXPANDING THE EXERCISE TO SCENARIO 2. FOR AN SPECIFIC DATE RETRIEVE ALL THE OD PAIRS

In [6]:
# Install the 'openpyxl' module
!pip install openpyxl

# Read the Excel file
file_path = r"C:\Users\Luis.ParraMorales\Documents\Python Code\Ana Rodriguez\Rail Fares - Web Scriping\NWEC - Origin and Destination List.xlsx"
ember_OD_names = pd.read_excel(file_path)

# Print the DataFrame
print(ember_OD_names.head())

  Operator          Origin                   Destination
0    Ember  Bridge of Earn                Bridge of Earn
1    Ember  Bridge of Earn  Corstorphine (Edinburgh Zoo)
2    Ember  Bridge of Earn       Dundee (Science Centre)
3    Ember  Bridge of Earn         Edinburgh (Haymarket)
4    Ember  Bridge of Earn  Edinburgh (St Andrew Square)


In [12]:
df_ember_names = pd.DataFrame(ember_OD_names) # Create a dataframe from the excel
df_ember_names = ember_OD_names[ember_OD_names['Operator']=='Ember'] #For now I'll just keep the rows for ember
# print(df_ember_names.head(16))

# URL for the JSON data
url_OD_codes_dictionary = "https://global.api.flixbus.com/cms/cities?language=en-gb&country=GB&limit=100"

# Set headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Data from requestor with headers
response_dictionary = requests.get(url_OD_codes_dictionary, headers=headers, verify=False)

# Initialize an empty list for the cities data
cities_data = []

# Check if the response was successful
if response_dictionary.status_code == 200:
    data_OD_codes = response_dictionary.json()  # Parse the JSON data
    
    # Extract the list of cities from the "result" key
    cities_data = data_OD_codes.get("result", [])
    
    if not cities_data:
        print("No cities data found in the response.")
else:
    print(f"Failed to retrieve data: {response_dictionary.status_code}")

# Process the cities data (this part is outside of the if block)
if cities_data:
    # List to store route info from the dictionary information
    route_ID_name = []

    # Extract the 'id' and 'name' from each entry in cities_data
    for entry in cities_data:
        route_id = entry.get("id")
        route_name = entry.get("name")

        # Append the route info as a dictionary to the list
        route_ID_name.append({
            "Location ID": route_id,
            "Location Name": route_name
        })

    # Create a DataFrame from the list containing all the route names and IDs
    df_dictionary_ID = pd.DataFrame(route_ID_name)

    # Display the DataFrame
    print(df_dictionary_ID)
else:
    print("No valid cities data to process.")

#output_path = r'C:\Users\Luis.ParraMorales\Documents\Python Code\Ana Rodriguez\Rail Fares - Web Scriping\FlixbusDictionary.xlsx'
#df_dictionary_ID.to_excel(output_path, index=False)


c:\Users\Luis.ParraMorales\AppData\Local\miniforge3\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'global.api.flixbus.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


    Location ID    Location Name
0          3848           London
1         14638       Manchester
2         43131          Bristol
3         14668       Birmingham
4         46891          Glasgow
..          ...              ...
75        51011  South Petherton
76        50891  North Petherton
77       121843         Saltford
78        51041        Wincanton
79        53421           Othery

[80 rows x 2 columns]


In [14]:
#CHECKING ALL THE NAMES
#print(df_dictionary_ID['Location Name'].unique())
#print(df_dictionary_ID)

['London' 'Manchester' 'Bristol' 'Birmingham' 'Glasgow' 'Edinburgh'
 'Leeds' 'Cardiff' 'London Heathrow Airport' 'Nottingham'
 'Newcastle upon Tyne' 'Aberdeen' 'Leicester' 'Liverpool' 'Sheffield'
 'Dundee' 'Cambridge' 'London Gatwick Airport' 'Swansea'
 'Newport (South Wales)' 'Bristol Airport' 'Perth' 'Plymouth' 'Reading'
 'Manchester Airport' 'Middlesbrough' 'Stoke-on-Trent' 'Inverness'
 'Coventry' 'Exeter' 'York' 'Bradford' 'Sunderland' 'Preston'
 'Northampton' 'Bath' 'Peterborough' 'Bridgend' 'Stirling' 'Hull'
 'Taunton' 'Stevenage' 'Carlisle' 'Lancaster' 'Dover' 'Glasgow Airport'
 'Warrington' 'Livingston' 'Aviemore' 'Watford' 'Dunfermline' 'Pitlochry'
 'Guildford' 'Folkestone' 'Chesterfield' 'Port Talbot' 'Bridgwater'
 'Rugby (Warwickshire)' 'Woking' 'Amesbury' 'Huntingdon (Cambridgeshire)'
 'Holyhead' 'Penrith' 'Yeovil' 'Harthill' 'Glastonbury' 'Wellington'
 'Warminster' 'Frome' 'Shepton Mallet' 'Ilminster' 'Keynsham' 'Codford'
 'Street' 'Wells' 'South Petherton' 'North Petherto

In [23]:
#TESTING HOW MANY IDS HAS EACH LOCATION
print(df_dictionary_ID[df_dictionary_ID['Location Name']=='Newport (South Wales)'])

    Location ID          Location Name
19        47361  Newport (South Wales)


In [112]:
#Now I am merging the dataframe of the OD that I want to analyze by adding the ID of the origin and destination

#Merge and add data for the origins in df_ember_names based on the data of df_dictionary_ID 
#df_ember_names = df_ember_names.merge(df_dictionary_ID , how='left' , left_on = 'Origin' , right_on='Route Name')
#df_ember_names = df_ember_names.rename(columns={'Route ID':'Origin ID'}).drop(columns=['Route Name'])

#Same but for the destination
#df_ember_names = df_ember_names.merge(df_dictionary_ID , how='left' , left_on = 'Destination' , right_on='Route Name')
#df_ember_names = df_ember_names.rename(columns={'Route ID':'Destination ID'}).drop(columns=['Route Name'])

In [113]:
#print(df_ember_names.columns)